# LSTM Recurrent Neural Network

In [ ]:
# Vanilla libraries
import os
import sys
import pickle
from datetime import datetime

# 3rd-party libraries
import numpy as np
import tensorflow as tf
from tqdm import tqdm

# Helper library
from helpers import *

## File paths & pretrained models

In [ ]:
data_dir = 'datasets/'

saved_w2id = 'saved/word2id.pkl'
ids_matrix_file = 'saved/idsMatrix.npy'

tensorboard_dir = 'tensorboard/'
logdir = os.path.join(tensorboard_dir, 'log')
save_path = 'models/'
trained_model_path = os.path.join(save_path, 'model.ckpt')

### Word count and average words

In [ ]:
if not os.path.exists(saved_w2id):
    w2id = Word2ID(data_dir=data_dir)
    # Save it!
    with open(saved_w2id, mode='wb') as f:
        pickle.dump(obj=w2id, file=f)
else:
    f = open(saved_w2id, mode='rb')
    w2id = pickle.load(file=f)
    f.close()

In [ ]:
X_train = w2id.features
y_train = w2id.labels
print(X_train.shape, y_train.shape)

## Recurrent Neural Network

### Hyperparameters

In [ ]:
num_classes = 2
max_seq_len = w2id.max_seq_len
word_vector_size = 50  # 50-300

# Model
lstm_units = 128
keep_prob = 0.8

# Training
learning_rate = 1e-3
batch_size = 24
iterations = 10000
save_step = 50

## `tensorflow`'s Computational Graphs

In [ ]:
# Reset default graph
tf.reset_default_graph()

# Model's placeholders
X = tf.placeholder(tf.int32, shape=[batch_size, max_seq_len])
y = tf.placeholder(tf.float32, shape=[batch_size, num_classes])
y_true = tf.argmax(y, axis=1)

### Embedding lookup

In [ ]:
# helper.GLOVE_VECTORS
embedding = tf.nn.embedding_lookup(GLOVE_VECTORS, X)

### `LSTMCell`

In [ ]:
lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_units)
lstm_cell = tf.contrib.rnn.DropoutWrapper(cell=lstm_cell, output_keep_prob=keep_prob)

In [ ]:
outputs, state = tf.nn.dynamic_rnn(lstm_cell, embedding, dtype=tf.float32)

### Model

In [ ]:
W = tf.Variable(tf.truncated_normal(shape=[lstm_units, num_classes]), name='weight')
b = tf.Variable(tf.zeros(shape=[num_classes]), name='bias')

outputs = tf.transpose(outputs, [1, 0, 2])
layer = tf.gather(outputs, int(outputs.get_shape()[0]) - 1)
y_hat = tf.matmul(layer, W) + b
y_pred = tf.nn.softmax(y_hat)
y_pred_true = tf.argmax(y_pred, axis=1)

### `cross_entropy` `loss` & `optimizer`

In [ ]:
x_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_hat, labels=y)
loss = tf.reduce_mean(x_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(loss)

### Accuracy

In [ ]:
correct = tf.equal(y_true, y_pred_true)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

## Running `tf.Session()`

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)


## Tensorboard

In [ ]:
# Saver and Writer
saver = tf.train.Saver()
writer = tf.summary.FileWriter(logdir=logdir, graph=sess.graph)

# Tensorboard summary
tf.summary.scalar('loss', loss)
tf.summary.scalar('accuracy', accuracy)
tf.summary.histogram('weight', W)
tf.summary.histogram('bias', b)
merged = tf.summary.merge_all()

# Load pretrained or create save_path
if tf.gfile.Exists(save_path):
    if len(os.listdir(save_path)) > 1:
        saver.restore(sess=sess, save_path=tf.train.latest_checkpoint(save_path))
else:
    tf.gfile.MakeDirs(save_path)

## Training!!!

In [ ]:
# Clocking the time taken so far...
train_start = datetime.now()
for i in range(iterations):
    # Next batchwriter = tf.summary.FileWriter(logdir=logdir, graph=sess.graph)
    X_batch, y_batch = w2id.next_batch(batch_size=batch_size)
    feed_dict = {X: X_batch, y: y_batch}
    sess.run(train_step, feed_dict=feed_dict)
    if i % save_step == 0:
        # Save
        saver.save(sess=sess, save_path=trained_model_path, global_step=i)
        # Add summary
        summary = sess.run(merged, feed_dict=feed_dict)
        writer.add_summary(summary, global_step=i)
    sys.stdout.write('\r{:,} of {:,}\tTime taken: {}'.format(i+1, iterations, datetime.now()-train_start))

# Close the FileWriter
writer.close()

## Testing!!!

In [ ]:
n_tests = 2
for i in range(n_tests):
    X_batch, y_batch = w2id.next_batch(batch_size=batch_size)
    feed_dict = {X: X_batch, y: y_batch}
    acc = sess.run(accuracy, feed_dict=feed_dict)
    print('Accuracy {:,} = {:.2%}'.format(i+1, acc))

In [ ]:
def test(text):
    text_embedding = w2id.get_embeding_ids(text)
    sentiment = sess.run(y_pred, feed_dict={X: text_embedding})[0]
    return np.argmax(sentiment)